In [19]:
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 openai



In [20]:
!pip install torch==2.3.0 torchvision==0.18.0 --upgrade --quiet
!pip install --upgrade transformers accelerate safetensors --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu124 requires torch==2.5.1, but you have torch 2.3.0 which is incompatible.


Need to write models that will generate data

In [ ]:
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch


## Need to create environment

In [21]:
huggin_api_key = userdata.get("HF_TOKEN")
login(huggin_api_key, add_to_git_credential=True)


## MODELS THAT I'LL TRY FROM HF

In [22]:
llama='meta-llama/Meta-Llama-3.1-8B-Instruct'
qwen_alibaba='Qwen/Qwen2-7B-Instruct'

## Now I need to handle prompts

In [23]:
system_prompt="You are data generation assistant that produces realistic,structured synthetic datasets for multi-modal multiagent AI system.Your task is to simulate realistic interactions between agents and users across tabular data,text, time series and structured data formats. "



user_prompt=""" Generate 50 telemedicine sessions. Each session object must include :
  - session_id (UUID)
  - patient_id, doctor_id
  - start_time, end_time
  - turns: list of {timestamp, speaker: "Patient"|"Doctor", content}
  - vitals_timeseries: list of {timestamp, heart_rate, blood_pressure_systolic, blood_pressure_diastolic, temperature, respiratory_rate}
  - summary: {avg_heart_rate, min_heart_rate, max_temperature, abnormal_readings_count}
  - conclusion: {diagnosis, recommendations: […]}

 """


In [24]:
def data_format(format):
  if format=='JSON':
    additional_prompt="The output will be only in JSON format."
  elif format=='CSV':
    additional_prompt="The output will be only in CSV format."
  else:
    additional_prompt="The output will be only in MarkDown format."
  return additional_prompt

In [25]:
def model_messages(format):
  messages=[{'role':'system','content':system_prompt},{'role':'user','content':user_prompt+data_format(format)}]

  return messages

## MODELS

### First i will try quantization

In [26]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [27]:
def generate_model(messages,model_type):
  tokenizer=AutoTokenizer.from_pretrained(model_type)
  inputs=tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to("cuda")
  streamer=TextStreamer(tokenizer)
  model = AutoModelForCausalLM.from_pretrained(model_type, device_map="auto", quantization_config=quant_config)
  outputs=model.generate(inputs,streamer=streamer,max_new_tokens=2000)
  response = tokenizer.decode(outputs[0])

  return response


In [28]:
def generate_dataset(model_choice, format):
    if model_choice=='meta':
      model_type=llama
    else:
      model_type=qwen_alibaba

    messages = model_messages(format)
    dataset = generate_model(messages, model_type)
    return dataset

## Now i will create Interface

In [29]:
%pip install -q gradio

In [30]:
import gradio as gr

model_choices = ['meta', 'qwen']
format_choices = ['JSON', 'CSV', 'MarkDown']

iface = gr.Interface(
    fn=generate_dataset,
    inputs=[
        gr.Dropdown(model_choices, label="Select Model"),
        gr.Dropdown(format_choices, label="Select Format")
    ],
    outputs=gr.Textbox(label="Generated Dataset"),
    title="Synthetic Telemedicine Dataset Generator"
)

In [ ]:
iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e9f8cea99092f76a97.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are data generation assistant that produces realistic,structured synthetic datasets for multi-modal multiagent AI system.Your task is to simulate realistic interactions between agents and users across tabular data,text, time series and structured data formats.<|eot_id|><|start_header_id|>user<|end_header_id|>

Generate 50 telemedicine sessions. Each session object must include :
  - session_id (UUID)
  - patient_id, doctor_id
  - start_time, end_time
  - turns: list of {timestamp, speaker: "Patient"|"Doctor", content}
  - vitals_timeseries: list of {timestamp, heart_rate, blood_pressure_systolic, blood_pressure_diastolic, temperature, respiratory_rate}
  - summary: {avg_heart_rate, min_heart_rate, max_temperature, abnormal_readings_count}
  - conclusion: {diagnosis, recommendations: […]}

 The output will be only in JSON format.<|eot_id|><|start_header_id|>ass